In [ ]:
from Node import DS
from Tools import Func
from Tools import Config
import pandas as pd
import numpy as np
import os
import time
import operator
import sys

import re


def main():
    # set dataset and ground truth file path    
    filepath=r"..\dataset\caida"
    pattern='\w+.dat$'
    r=re.compile(pattern)
    filelist=list(filter(r.match,os.listdir(filepath)))    
        # dataset file list
    
    gr_file_name='caida_0_ground_truth.csv'
    gr_path=os.path.join(filepath,gr_file_name)
        # ground truth path
        
    # Top and Sketch parameter config
    w=256
    d=256
    size=2048
    Config.Set_default(w,d,size)
        # set width, depth, size of Sk, random seed of hash,e_max
    
    # Initialize Top, Sketch and Sk_head
    Sk_head=[DS.Head(0) for j in range(Config.depth)]
    Sketch=np.zeros((Config.depth,Config.width),dtype='int32')
    Top_dict=dict()
    e_min=DS.Tail("",1)
    
    # Stream processing
    start=time.time()
    for datafile in filelist[:1]:
        src_data=os.path.join(filepath,datafile)
        with open(src_data,'rb') as file:
             #以binary讀取，資料型態也為byte
            while True:
                e=str(file.read(13))
                if len(e)<13:
                    print('EOF')
                    break
                else:    
                    item=DS.Tail(e,1)
                    if Top_dict.get(item.ID):
                        # e in Top
                        Top_dict[item.ID]+=1
                    else:
                        if len(Top_dict)<Config.size:
                            Top_dict[item.ID]=1
                        else:
                            Func.UpdateSk(item,Sk_head,Sketch)
                if Config.e_max.count>e_min.count:
                    min_ele = min(Top_dict, key=Top_dict.get)
                        # 找dict中的最小key元素
                    e_min=DS.Tail(min_ele,Top_dict[min_ele])
                    Func.BringBack(e_min,Top_dict,Sk_head,Sketch)
    end=time.time()
    print("Top-{},Sketch:{}*{}".format(Config.size,Config.depth,Config.width))
    print("Execution time:{:8.3f} seconds.".format(end-start))
    Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
    
    # Element-Precision
    tp_set,precision=Func.Get_precision(gr_path,Top_dict)

    # Count ARE/AAE

    startx=time.time()
    top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,Top_dict,Sk_head,Sketch,4)
    endx=time.time()
    print("Find:{}, TP:{}, FP:{}".format(len(tp_set),tp,fp))
    print("Top_ARE: {:8.6f}".format(top_are))
    print("Top_AAE: {:8.6f}".format(top_aae))
    print("all_ARE: {:8.6f}".format(all_are))
    print("all_AAE: {:8.6f}".format(all_aae))
    print("Estimate time:{:8.3f} seconds.".format(endx-startx))    

    startx=time.time()
    top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,Top_dict,Sk_head,Sketch,5)
    endx=time.time()
    print("Find:{}, TP:{}, FP:{}".format(len(tp_set),tp,fp))
    print("Top_ARE: {:8.6f}".format(top_are))
    print("Top_AAE: {:8.6f}".format(top_aae))
    print("all_ARE: {:8.6f}".format(all_are))
    print("all_AAE: {:8.6f}".format(all_aae))
    print("Estimate time:{:8.3f} seconds.".format(endx-startx))      
    

if __name__=='__main__':
    main()

In [1]:
from Node import DS
from Tools import Func
from Tools import Config
import pandas as pd
import numpy as np
import os
import time
import operator
import sys

import re

# set dataset and ground truth file path    
filepath=r"..\dataset\caida"
pattern='\w+.dat$'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='caida_0_ground_truth.csv'
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path

# Top and Sketch parameter config
w=128
d=64
size=1024
Config.Set_default(w,d,size)
    # set width, depth, size of Sk, random seed of hash,e_max

# Initialize Top, Sketch and Sk_head
Sk_head=[DS.Head(0) for j in range(Config.depth)]
Sketch=np.zeros((Config.depth,Config.width),dtype='int32')
Top_dict=dict()
e_min=DS.Tail("",1)

# Stream processing
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'rb') as file:
         #以binary讀取，資料型態也為byte
        while True:
            e=str(file.read(13))
            if len(e)<13:
                print('EOF')
                break
            else:    
                item=DS.Tail(e,1)
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID]+=1
                else:
                    if len(Top_dict)<Config.size:
                        Top_dict[item.ID]=1
                    else:
                        Func.UpdateSk(item,Sk_head,Sketch)
            if Config.e_max.count>e_min.count:
                min_ele = min(Top_dict, key=Top_dict.get)
                    # 找dict中的最小key元素
                e_min=DS.Tail(min_ele,Top_dict[min_ele])
                Func.BringBack(e_min,Top_dict,Sk_head,Sketch)
end=time.time()
print("Top-{},Sketch:{}*{}".format(Config.size,Config.depth,Config.width))
print("Execution time:{:8.3f} seconds.".format(end-start))
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))

# Element-Precision
tp_set,precision=Func.Get_precision(gr_path,Top_dict)

# Count ARE/AAE

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,Top_dict,Sk_head,Sketch,4)
endx=time.time()
print("Find:{}, TP:{}, FP:{}".format(len(tp_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))    


startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,Top_dict,Sk_head,Sketch,6)
endx=time.time()
print("Find:{}, TP:{}, FP:{}".format(len(tp_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))    


EOF
Top-1024,Sketch:64*128
Execution time:  44.549 seconds.
Precision:   0.8789
lhh_are:1.0822278620877575,lhh_aae:122.5625
Find:900, TP:900, FP:124
Top_ARE: 0.039345
Top_AAE: 21.735556
all_ARE: 3.011094
all_AAE: 9.158466
Estimate time:   3.284 seconds.
lhh_are:6.1728366382213125,lhh_aae:82.0625
Find:900, TP:900, FP:124
Top_ARE: 0.039345
Top_AAE: 21.735556
all_ARE: 3.012941
all_AAE: 9.143775
Estimate time:   2.903 seconds.


In [ ]:
# compare top k
import pandas as pd
import os

filepath=r"..\dataset\caida"
gr_file_name='caida_0_ground_truth.csv'
gr_path=os.path.join(filepath,gr_file_name)

grtruth=pd.read_csv(gr_path)
grtruth['Element'] = grtruth['Element'].astype(str)
gt_dict=dict(grtruth[:Config.size].values.tolist())
gt_dict

tp_set=Top_dict.keys() & gt_dict.keys()
print("Top_dict vs. gt_dict:")
smaller=0
bigger=0
for item in tp_set:
    if Top_dict[item]<gt_dict[item]:
        smaller+=1
        print("item:{}:".format(item))
        print('count:{},{}'.format(Top_dict[item],gt_dict[item]))
    elif Top_dict[item]>gt_dict[item]:
        bigger+=1
        #print("item:{}:".format(item))
        #print('count:{},{}'.format(Top_dict[item],gt_dict[item]))        
print(smaller)


In [ ]:
# method 1